In [1]:
import pandas as pd
import numpy as np

In [62]:
filename = 'shrinkage_fc0'
file_type = 'swell'
trorte = 'training'

In [63]:
data = pd.read_csv("dataset/{}_{}_{}.csv".format(filename, file_type, trorte))
info = pd.read_csv("dataset/lmfit_data.csv")
data.shape, info.shape

((2640, 16), (250, 9))

In [64]:
data_property = list(data.columns)
print(len(data_property))

16


In [65]:
#info.dropna(inplace=True)
#info.shape

In [66]:
filenames = np.asarray(info["File"].drop_duplicates().reset_index(drop=True))
filenames.shape

(250,)

In [67]:
select_property = ['File', 'wb', 'ab', 't dry']

In [68]:
def shrinkage(t, wb, ab, epsilon_inf, t_sw, t_au, m, n):
    return epsilon_inf * (wb) / (1 + ab) * (wb / 4 / (1 + (t_sw / t)**m) - (1 - wb / 4) / (1 + (t_au / t)**n ))

In [69]:
def fitting_curve2_full(filenames, data, info, num_data):
    for i in range(len(info)):
        try:
            cement = data[data["File"]==filenames[i]]
            wb = cement['wb'].unique()
            ab = cement['ab'].unique()
            param = info[info["File"]==filenames[i]]
            epsilon_inf = float(param['epsilon_inf'].unique())
            t_sw = float(param['t_sw'].unique())
            t_au = float(param['t_au'].unique())
            m  = float(param['m'].unique())
            n  = float(param['n'].unique())
            time = np.asarray(cement["t"])
            if time.size == 0 or np.isnan(np.sum(time)):
                continue
            time_gen = np.linspace(np.min(time), np.max(time), num_data)
            shrinkage_gen = np.empty_like(time_gen)
            for j in range(len(time_gen)):
                shrinkage_gen[j] = shrinkage(time_gen[j], wb, ab, epsilon_inf, t_sw, t_au, m, n)
            #print(shrinkage_gen)
            time_gen = np.reshape(time_gen, (len(time_gen), 1))
            shrinkage_gen = np.reshape(shrinkage_gen, (len(shrinkage_gen), 1))
            #spec = np.asarray(cement[data_property[: len(data_property) - 2]])[0]
            spec = np.asarray(cement[select_property])[0]
            spec = np.reshape(spec, (1, len(spec)))
            data_gen = np.repeat(spec, num_data, axis=0)
            ml_data = np.concatenate([data_gen, time_gen, shrinkage_gen], axis=1)
            print(i)
            if i==1:
                ml_dataset = ml_data
            else:
                ml_dataset = np.append(ml_dataset, ml_data, axis=0)
        except:
            continue
            
    select_property.append('t')
    select_property.append('shrinkage strain') 
    df_dataset = pd.DataFrame(ml_dataset, columns=select_property)
    
    return df_dataset

In [70]:
def fitting_curve2(filenames, data, info, min_val, max_val, num_data):
    for i in range(len(info)):
        cement = data[data["File"]==filenames[i]]
        wb = cement['wb'].unique()
        ab = cement['ab'].unique()
        param = info[info["File"]==filenames[i]]
        time = np.asarray(cement["t"])
        if time.size == 0:
            #print(i)
            #print(cement["dt"])
            continue
        if np.max(time) < max_val:
            continue
        time_gen = np.linspace(min_val, max_val, num_data)
        shrinkage_gen = np.empty_like(time_gen)
        for j in range(len(time_gen)):
            shrinkage_gen[j] = shrinkage(time_gen[j], param['epsilon_inf'], param['a1'],param['a2'],param['c1'],param['c2'],param['d1'],param['d2'],wb,ab,param['m'],param['n'])
        time_gen = np.reshape(time_gen, (len(time_gen), 1))
        shrinkage_gen = np.reshape(shrinkage_gen, (len(shrinkage_gen), 1))
        #spec = np.asarray(cement[data_property[: len(data_property) - 2]])[0]
        spec = np.asarray(cement[select_property][0])
        print(spec.shape)
        
        spec = np.reshape(spec, (1, len(spec)))
        data_gen = np.repeat(spec, num_data, axis=0)
        ml_data = np.concatenate([data_gen, time_gen, shrinkage_gen], axis=1)
        print(i)
        if i==1:
            ml_dataset = ml_data
        else:
            ml_dataset = np.append(ml_dataset, ml_data, axis=0)
    
    select_property.append('t')
    select_property.append('shrinkage strain')         
    df_dataset = pd.DataFrame(ml_dataset, columns=select_property)
    
    return df_dataset

In [71]:
num_data = 200
fc2_full = fitting_curve2_full(filenames, data, info, num_data)
print(fc2_full.shape)
fc2_full.to_csv('shrinkage_fc2_{}_{}_{}.csv'.format(file_type, trorte, num_data), index=False)

1
2
4
14
30
31
33
34
35
38
40
42
43
44
45
46
47
48
49
51
52
53
54
56
58
59
61
62
63
64
65
67
68
69
70
73
79
80
81
82
83
84
85
88
92
94
99
101
102
103
104
105
106
107
108
110
111
112
113
114
116
117
121
127
128
131
132
133
136
137
140
141
145
156
157
158
159
160
162
164
168
170
171
172
174
175
180
183
184
185
186
189
190
191
209
210
211
213
214
215
216
217
218
219
221
222
223
226
228
230
231
232
233
234
235
236
237
238
240


/home/vgf3011/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  
/home/vgf3011/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


241
242
243
244
245
246
247
248
(25400, 6)
